In [1]:
from google.cloud import bigquery
bq = bigquery.Client(project="cs544-s23-376219")
job_config = bigquery.QueryJobConfig(use_query_cache=False)

In [2]:
#q1
query_q1 = """
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane' AND state_fips_code = '55'
"""
result_q1 = bq.query(query_q1, job_config=job_config)
result_q1.to_dataframe().loc[0,"geo_id"]

'55025'

In [3]:
#q2
query_q2 = """
SELECT state_fips_code, COUNT(*) as counties_count
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY counties_count DESC
LIMIT 5
"""
result_q2 = bq.query(query_q2, job_config=job_config)
df = result_q2.to_dataframe()
keys = list(df.loc[:, "state_fips_code"])
values = list(df.loc[:, "counties_count"])

dict(zip(keys, values))

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [4]:
#q3
q1_bill = result_q1.total_bytes_billed
q2_bill = result_q2.total_bytes_billed
cost = 5  

cost_q1 = (q1_bill / 1e12) * cost
cost_q2 = (q2_bill / 1e12) * cost

{'q1': cost_q1, 'q2': cost_q2}

{'q1': 5.2428799999999995e-05, 'q2': 5.2428799999999995e-05}

In [5]:
#q4

gcs_uri = "gs://bencolemanbucket/hdma-wi-2021.parquet"
project_id = bq.project

dataset_id = f"{project_id}.p7"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
bq.create_dataset(dataset, exists_ok=True)

table_id = f"{project_id}.p7.hdma"
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

load_job = bq.load_table_from_uri(gcs_uri, table_id, job_config=job_config)
load_job.result()

table = bq.get_table(table_id)
[ds.dataset_id for ds in bq.list_datasets("cs544-s23-376219")]

['p7']

In [6]:
#q5
query_q5 = """
SELECT c.county_name, COUNT(*) as loan_count
FROM cs544-s23-376219.p7.hdma as h
JOIN bigquery-public-data.geo_us_boundaries.counties as c
ON h.county_code = c.county_fips_code
WHERE h.county_code IS NOT NULL
GROUP BY c.county_name
ORDER BY loan_count DESC
LIMIT 10
"""

df = {}
result_q5 = bq.query(query_q5)
for row in result_q5: 
    df[row["county_name"]] = row["loan_count"]
df


{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [7]:
#q6

config = bigquery.ExternalConfig("GOOGLE_SHEETS")
config.source_uris = ["https://drive.google.com/open?id=1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA"]
config.autodetect = True

table = bigquery.Table("cs544-s23-376219.p7.loan_applications")
table.external_data_configuration = config

table = bq.create_table(table, exists_ok=True)

query_q6 = f"""
SELECT COUNT(*) as num_applications
FROM cs544-s23-376219.p7.loan_applications
WHERE Income = 120000
"""

results_q6 = bq.query(query_q6).to_dataframe()
results_q6['num_applications'][0]

7

In [8]:
#q7
query_q7 = f"""
SELECT c.county_name, COUNT(*) as num_applications
FROM cs544-s23-376219.p7.loan_applications as l
JOIN bigquery-public-data.geo_us_boundaries.counties as c
ON ST_CONTAINS(c.county_geom, ST_GEOGPOINT(l.Longitude, l.Latitude))
WHERE c.state_fips_code = '55'
GROUP BY c.county_name
ORDER BY num_applications DESC
"""

results_q7 = bq.query(query_q7).to_dataframe()
county_application_counts = dict(zip(results_q7['county_name'], results_q7['num_applications']))
county_application_counts


{'Dane': 26,
 'Door': 5,
 'Brown': 3,
 'Walworth': 2,
 'Columbia': 2,
 'Bayfield': 2,
 'Jefferson': 2,
 'Adams': 1,
 'La Crosse': 1,
 'Sheboygan': 1,
 'Barron': 1,
 'Iron': 1,
 'Waukesha': 1,
 'Iowa': 1,
 'Green Lake': 1,
 'Kewaunee': 1,
 'Milwaukee': 1,
 'Sauk': 1,
 'Monroe': 1,
 'Oneida': 1}

In [9]:
#q8
query = f"""
CREATE OR REPLACE MODEL `cs544-s23-376219.p7.loan_prediction_model`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['loan_amount']) AS
SELECT
  loan_amount,
  income,
  loan_term
FROM
  `cs544-s23-376219.p7.hdma`

"""

bq.query(query).result()

query_q8 = f"""
SELECT
  r2_score
FROM
  ML.EVALUATE(MODEL `cs544-s23-376219.p7.loan_prediction_model`,
  (SELECT income, loan_term, loan_amount
  FROM cs544-s23-376219.p7.hdma)
  )
"""

results_q8 = bq.query(query_q8).to_dataframe()
results_q8['r2_score'][0]

0.29165346922392255

In [10]:
#q9
query_q9 = f"""
SELECT
  processed_input,
  weight
FROM
  ML.WEIGHTS(MODEL `cs544-s23-376219.p7.loan_prediction_model`)
"""

results_q9 = bq.query(query_q9).to_dataframe()
income_weight = results_q9.loc[results_q9['processed_input'] == 'income', 'weight'].values[0]
income_weight

0.8046310253990144

In [11]:
#q10
loan_weight = results_q9.loc[results_q9['processed_input'] == 'loan_term', 'weight'].values[0]
intercept_weight = results_q9.loc[results_q9['processed_input'] == '__INTERCEPT__', 'weight'].values[0]

query_q10 = f"""
SELECT income, loan_amount,
       {intercept_weight} + {income_weight} * Income + {loan_weight} * 360 as predicted_amount
FROM cs544-s23-376219.p7.loan_applications
"""

results_q10 = bq.query(query_q10).to_dataframe()

amounts = results_q10["loan_amount"]
predicted = results_q10['predicted_amount']
greater = 0
for i in range(len(results_q10)):
    if amounts[i] > predicted[i]:
        greater += 1
greater / len(results_q10)

0.5980392156862745